Data manipulation to consolidate the file

In [1]:
import modin.pandas as pd
import numpy as np

dfe = pd.read_csv('./oldData/3cixty_cotedazur_events.csv', dtype=str)

dfe['label'] = dfe['label'].apply(lambda x: 'Event name: ' + str(x) if pd.notna(x) else x)
dfe['description'] = dfe['description'].apply(lambda x: 'Event description: ' + str(x) if pd.notna(x) else x)
dfe['category'] = dfe['category'].apply(lambda x: 'Event category: ' + str(x) if pd.notna(x) else x)
dfe['subject'] = dfe['subject'].apply(lambda x: 'Event subject: ' + str(x) if pd.notna(x) else x)
dfe['placeLabel'] = dfe['placeLabel'].apply(lambda x: 'Event place: ' + str(x) if pd.notna(x) else x)
dfe['placeLocality'] = dfe['placeLocality'].apply(lambda x: 'Event place location: ' + str(x) if pd.notna(x) else x)
dfe['timeBegin'] = dfe['timeBegin'].apply(lambda x: 'Event time begin: ' + str(x) if pd.notna(x) else x)
dfe['timeEnd'] = dfe['timeEnd'].apply(lambda x: 'Event time end: ' + str(x) if pd.notna(x) else x)

dfe = dfe.groupby('event', as_index=False).agg({
    'label': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'description': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'category': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'subject': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'placeLabel': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'placeLocality': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'timeBegin': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'timeEnd': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else ''
})

dfe.pop('event')

print(dfe.head())

# Save the result to a new CSV file
dfe.to_csv('./data/events.csv', index=False)

dfp = pd.read_csv('./oldData/3cixty_cotedazur_places.csv', dtype=str)

dfp['label'] = dfp['label'].apply(lambda x: 'Place name: ' + str(x) if pd.notna(x) else x)
dfp['description'] = dfp['description'].apply(lambda x: 'Place description: ' + str(x) if pd.notna(x) else x)
dfp['businessTypeLabel'] = dfp['businessTypeLabel'].apply(lambda x: 'Place business type: ' + str(x) if pd.notna(x) else x)
dfp['locality'] = dfp['locality'].apply(lambda x: 'Place location: ' + str(x) if pd.notna(x) else x)

dfp = dfp.groupby('place', as_index=False, sort=False).agg({
    'label': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'description': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'businessTypeLabel': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else '',
    'locality': lambda x: x.dropna().iloc[0] if x.dropna().size > 0 else ''
})

dfp.pop('place')

print(dfp.head())

dfp.to_csv('./data/places.csv', index=False)


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-12-23 15:56:26,356	INFO worker.py:1673 -- Started a local Ray instance.


                                               label  \
0  Event name: Séjour : le Brame du Cerf à la Rés...   
1                    Event name: MEDIATHEQUE D'AURON   
2  Event name: ALL FRENCH RIVIERA FOR ONE DAY SIG...   
3  Event name: Conférence "Demain l'Humain" - Dem...   
4  Event name: Visite privée : excursion d'une de...   

                                         description  \
0  Event description: Séjour VIP  Tout le cérémo...   
1                                                      
2  Event description: <p><strong>Your guide will ...   
3  Event description: Lieu: <strong>Palais des Co...   
4  Event description: Découvrez la richesse des s...   

                                            category  \
0                  Event category: Nature et détente   
1  Event category: Activités sportives, culturell...   
2                    Event category: Circuits privés   
3                                                      
4                    Event category: Circuits 

In [1]:
import glob
from llama_index import VectorStoreIndex, ServiceContext, Document, StorageContext
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, StorageContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
import chromadb
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext


In [2]:
system_prompt = (
    "You are a knowledgeable Tourism Assistant designed to provide visitors with "
    "information, recommendations, and tips for exploring and enjoying their destination. "
    "The assistant is familiar with a wide range of topics including historical sites, "
    "cultural events, local cuisine, accommodations, transportation options, and hidden gems. "
    "It offers up-to-date and personalized information to help tourists make the most of their trip."
)

service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo-1106", temperature=0.6, system_prompt=system_prompt), 
                                               embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name='dangvantuan/sentence-camembert-large', 
                                                                                                    model_kwargs = {'device': 'cuda:0'})))


# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("tourism")

# assign chroma as the vector_store to the context
storage_context = StorageContext.from_defaults(vector_store=ChromaVectorStore(chroma_collection=chroma_collection))

No sentence-transformers model found with name C:\Users\chris/.cache\torch\sentence_transformers\dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


If we need to compute the index:

In [8]:
documents = []

# Get a list of all CSV files in the directory
csv_files = glob.glob('./data/*.csv')

for file in csv_files:
    df = pd.read_csv(file, dtype=str, parse_dates=True)

    # Convert the DataFrame into a list of Document objects
    docs = [Document(doc_id=str(i), text=row.to_string()) for i, row in df.iterrows()]

    # Add the documents to the list
    documents.extend(docs)

batch_size = 5461  # Maximum batch size
for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    # Now add the batch to the index
    index = VectorStoreIndex.from_documents(batch, service_context=service_context, storage_context=storage_context, show_progress=True, use_asynch=True)
    storage_context.persist(persist_dir=f"./chroma_db")
#storage_context.persist(persist_dir=f"./chroma_db")

Generating embeddings: 100%|██████████| 3890/3890 [00:32<00:00, 120.39it/s]


if the index is already there:

In [3]:
index = VectorStoreIndex.from_vector_store(ChromaVectorStore(chroma_collection=chroma_collection), storage_context=storage_context)

In [6]:
print(index.as_query_engine().query(input("Enter your query: ")))

KeyboardInterrupt: Interrupted by user